# UAV PQC Scheduler – Fused Dataset AnalysisThis notebook fuses GCS summary rows, telemetry windows, and power traces to analyzegraceful degradation trade-offs (energy, latency, temperature) per suite and condition.

In [ ]:
import json, os, pathlib, math, time, itertoolsfrom pathlib import Pathimport pandas as pdimport numpy as npimport plotly.express as pxROOT = Path(r"c:\Users\burak\Desktop\research").resolve()OUTDIR = ROOT / "logs/auto/gcs"

In [ ]:
# Load GCS summary rows (written by gcs_scheduler.write_summary)summary_path = OUTDIR / "summary.csv"summary = pd.read_csv(summary_path) if summary_path.exists() else pd.DataFrame()print("summary rows:", len(summary))# Load telemetry samples snapshot (if exported in combined workbook or JSON)telemetry_json = OUTDIR / "telemetry_samples.json"if telemetry_json.exists():    telemetry = pd.read_json(telemetry_json)else:    telemetry = pd.DataFrame()display(summary.head())

In [ ]:
# Feature engineering helpersdef battery_bin(pct):    try:        v = float(pct)    except Exception:        return None    bins = [(90,100),(80,90),(70,80),(60,70),(50,60),(40,50),(30,40),(20,30),(10,20),(0,10)]    for lo,hi in bins:        if v >= lo and v <= hi:            return f"{lo}-{hi}"    return Nonedef temp_slope(series, dt_s=1.0):    if len(series) < 2:        return np.nan    return (series.iloc[-1] - series.iloc[0]) / max(1e-3, dt_s*(len(series)-1))

In [ ]:
# Minimal fusion example (extend with power traces and proxy metrics as needed)df = summary.copy()if not df.empty:    # Example: compute battery bins if battery_pct present; fall back to placeholders    if 'battery_pct' in df.columns:        df['battery_bin'] = df['battery_pct'].apply(battery_bin)    else:        df['battery_bin'] = None    # Normalize energy and power names if present    for c in ['power_energy_j','energy_j']:        if c in df.columns:            df['energy_j'] = df[c]    for c in ['power_avg_w','avg_power_w']:        if c in df.columns:            df['avg_power_w'] = df[c]display(df.head())

In [ ]:
# Quick view: energy by suiteif not df.empty and 'suite' in df.columns and 'energy_j' in df.columns:    fig = px.box(df, x='suite', y='energy_j', title='Energy (J) by Suite', points='all')    fig.show()else:    print('Add runs to summary.csv to visualize.')